# 3-DaskDataframe

This notebook demonstrates how to read and process a tabular datafile with the [Dask](https://docs.dask.org/en/stable/dataframe.html) Dataframe library. A Dask DataFrame is a large parallel DataFrame composed of many smaller Pandas DataFrames. Dask can handle datasets that are large than the available memory (out-of-core) and process them in parallel on multiple cores.

Author: Peter W. Rose (pwrose@ucsd.edu)

In [1]:
import os
import pandas as pd
from dask.distributed import Client
import dask.dataframe as dd
import time

If LOCAL_SCRATCH_DIR environment variable is not set, this notebook accesses the ../data directory for temporary files.

In [2]:
DATA_DIR = os.getenv("LOCAL_SCRATCH_DIR", default="../data")
filename = os.path.join(DATA_DIR, "gene_info.tsv")
file_size = f"{os.path.getsize(filename)/1E9:.1f}"
RESULTS_DIR = "results"
os.makedirs(RESULTS_DIR, exist_ok=True)

### Setup Benchmark

The ```n_cores``` and ```file_format``` parameter are used for benchmarking ([see](7-ParallelEfficiency.ipynb)). 
The cell below has been [parameterized](https://papermill.readthedocs.io/en/latest/usage-parameterize.html#jupyterlab-3-0) as input parameters for [papermill](https://papermill.readthedocs.io/en/latest/index.html).

In [3]:
n_cores = 0 # default -> uses all available cores
file_format = "csv"
#file_format = "parquet"

In [4]:
start = time.time()

### Read Data

In [5]:
if n_cores > 0:
    # use n_cores for benchmarking
    client = Client(n_workers=n_cores, threads_per_worker=1)
else:
    # use all available cores
    client = Client()

In [6]:
# read only specified columns and rows
column_names = ["GeneID", "Symbol", "Synonyms", "description", "type_of_gene", "#tax_id", "chromosome"]
filters=[[("type_of_gene", "==", "protein-coding")]]

if file_format == "csv":
    filename = os.path.join(DATA_DIR, "gene_info.tsv")
    genes = dd.read_csv(filename, usecols=column_names, dtype=str, sep="\t")
    genes = genes.query("type_of_gene == 'protein-coding'")
elif file_format == "parquet":
    filename = os.path.join(DATA_DIR, "gene_info.parquet")
    genes = dd.read_parquet(filename, columns=column_names, filters=filters, engine="pyarrow")    
else:
    print("invalid file format")
    
print("Filename:", filename)

genes = genes.rename(columns={"#tax_id": "tax_id"})

Filename: /scratch/pwrose/job_22855019/gene_info.tsv


### Process Data

In [7]:
groups = genes.groupby("tax_id").size().reset_index()
groups.columns = ["tax_id", "count"]
groups = groups.sort_values("count", ascending=False)

Convert Dask to Pandas dataframe (this triggers the computation)

In [8]:
# groups = groups.compute(scheduler="processes") # use if Dask client is not instantiated
groups = groups.compute()

In [9]:
# terminate the Dask processes
client.close()

### Display Results

#### Number of human protein-coding genes (tax_id = 9606)

In [10]:
groups.query("tax_id == '9606'")

,tax_id,count
1542,9606,20648


#### Top 5 organisms with the most protein-coding genes

In [11]:
groups.head()

,tax_id,count
727,4565,104037
512,3708,90975
7544,90675,82686
18134,412133,72290
7720,94328,68154


In [12]:
end = time.time()

In [13]:
if n_cores > 0:
    df = pd.DataFrame([{"cores": n_cores, "time": end-start, "size": file_size, "format": file_format, "dataframe": "Dask"}])
    output_file = f"3-DaskDataframe_{file_size}_{file_format}_{n_cores}.csv"
    df.to_csv(os.path.join(RESULTS_DIR, output_file), index=False)

In [14]:
print(f"Dask: {end - start:.1f} sec.")

Dask: 18.6 sec.
